In [ ]:
import config as c
import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder
import os
from IPython.core.display import HTML
import time
import random
from lxml import etree
import json

# Login

In [ ]:
s = requests.Session()
s.post('https://transkribus.eu/TrpServer/rest/auth/login', data={"user": c.login, "pw":c.password})

## List collections

In [ ]:
collections = s.get('https://transkribus.eu/TrpServer/rest/collections/list')

In [ ]:
json.loads(collections.content)

In [ ]:
docs = s.get('https://transkribus.eu/TrpServer/rest/collections/{id}/list')

In [ ]:
json.loads(docs.content)

In [ ]:
ids = [obj["docId"] for obj in json.loads(docs.content)]
print(ids)

## Get Transkribus ID from sesamid

In [ ]:
sesamids = ['']

In [ ]:
ids = [obj["docId"] for obj in json.loads(docs.content) if obj["title"] in sesamids]

In [ ]:
len(ids)

## Delete from collection

In [ ]:
ids = []

In [ ]:
for id in ids:
    print(id)
    r = s.delete('https://transkribus.eu/TrpServer/rest/collections/{collId}/' + str(id))
    print(r.status_code)
    time.sleep(1)

# Upload to collection

In [ ]:
uploadObj = {
    "md": {
        "title": "EXAMPLE-ID"
    },
    "pageList": {"pages": [
        {
            "fileName": "0001.jpg",
            "pageNr": 1
        },
        {
            "fileName": "0002.jpg",
            "pageNr": 2
        }
    ]}
}

In [ ]:
uploadObj

In [ ]:
headers = {'Content-type': 'application/json'}
cont = s.post('https://transkribus.eu/TrpServer/rest/uploads?collId={id}', json=uploadObj, headers=headers)

In [ ]:
cont.content

In [ ]:
# parse and get upload ID
response = etree.fromstring(cont.content)
uploadId = response.xpath('//uploadId/text()')[0]
print(uploadId)

In [ ]:
# loop through files
for file in ['0001.jpg', '0002.jpg']:
    print(file)
    files = {file: open(file, 'rb')}
    
    mp_encoder = MultipartEncoder(
    fields={
        # plain file object, no filename or mime type produces a
        # Content-Disposition header with just the part name
        'img': (file, open(file, 'rb'), 'application/octet-stream'),
        }
    )
    
    cont = s.put('https://transkribus.eu/TrpServer/rest/uploads/' + uploadId, data=mp_encoder, headers={'Content-Type': mp_encoder.content_type})
    print(cont.content)
    time.sleep(random.randint(0,2))